In [ ]:
import numpy as np
import os
from keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input,decode_predictions
from keras.layers import Dense, Activation, Flatten, concatenate
from keras.layers import Input
from keras.models import Model
from tensorflow.keras.utils import load_img
from keras.utils import np_utils
from sklearn.model_selection import train_test_split

# Start Problem 1

In [ ]:
#######dataset contains diff. flower dataset folders ##### 
##### ( due to low resource I have used only three flower dataset, we can use more also, only need to put in that dataset folder)


############## Now Load dataset ##############################

def load_dataset(dataset='dataset'):
    PATH = os.getcwd()
    # Define data path
    PATH = "/content/drive/My Drive/Colab Notebooks"
    data_path = PATH + '/'+ dataset    
    data_dir_list = os.listdir(data_path)
    class_names = []
    img_data_list=[]
    labels = []
    for dataset in data_dir_list:
        class_names.append(dataset)
        img_list=os.listdir(data_path+'/'+ dataset)
        print ('Loaded the images of dataset-'+'{}\n'.format(dataset))
        lb = len(class_names)-1
        for img in img_list:
            img_path = data_path + '/'+ dataset + '/'+ img
            img = load_img(img_path, target_size=(224, 224))
            x = image.img_to_array(img)
            x = np.expand_dims(x, axis=0)            
            x = x/255            
            img_data_list.append(x)
            labels.append(lb)
    img_data = np.array(img_data_list)    
    img_data=np.rollaxis(img_data,1,0)    
    img_data=img_data[0]
    print (img_data.shape)
    labels=np.array(labels)
    print(labels.shape)
    num_classes = len(class_names)
    return img_data, labels, class_names
img_data, labels, class_names = load_dataset(dataset='dataset')

############## it returns img_data, labels and class_names ##############################



In [ ]:
############## Now creates pseduo labels for the dataset ##############################

def rotate_img(img, rot):
    if rot == 0:  # 0 degrees rotation
        return img
    elif rot == 90:  # 90 degrees rotation
        return np.flipud(np.transpose(img, (1, 0, 2)))
    elif rot == 180:  # 90 degrees rotation
        return np.fliplr(np.flipud(img))
    elif rot == 270:  # 270 degrees rotation / or -90
        return np.transpose(np.flipud(img), (1, 0, 2))

def rot_data(img_data):
    rot_img_data = []
    rot_labels = []
    for img in img_data:
        img_1 = rotate_img(img,0)
        rot_img_data.append(img_1)
        rot_labels.append(0)
        img_2 = rotate_img(img,90)
        rot_img_data.append(img_2)
        rot_labels.append(1)
        img_3 = rotate_img(img,180)
        rot_img_data.append(img_3)
        rot_labels.append(2)
        img_4 = rotate_img(img,270)
        rot_img_data.append(img_4)
        rot_labels.append(3)
        #print(rot_img_data.shape,rot_labels.shape)
        #break
    rot_img_data = np.array(rot_img_data)
    #rot_labels = np_utils.to_categorical(rot_labels, 4)    
    return rot_img_data, rot_labels

rot_img_data, rot_labels = rot_data(img_data)

##############  it returns rotation_img_data, pseduo labels (0,1,2,3)  ########################

In [ ]:
rot_labels = np.array(rot_labels)
#### we can plot the data and see by ourselves
import matplotlib.pyplot as plt
fig = plt.figure(1,figsize=(12,12))
for i in range(4):
  plt.subplot(4,4,i+1)
  plt.tight_layout()
  #x[i] = x[i][:,:,::-1] # converting BGR to RGB
  plt.imshow(rot_img_data[i][:,:,::-1], interpolation='none')
  plt.title("class_label: {}".format(rot_labels[i]))
  plt.xticks([])
  plt.yticks([])
plt

In [ ]:
# convert class psuedo labels to on-hot encoding
X_psuedo_data = rot_img_data
Y_psuedo_labels = np_utils.to_categorical(rot_labels, 4)  #  labels: (0,90,180,270) (0,1,2,3) ==> 4 classes
num_classes = (Y_psuedo_labels.shape)[1]
print(X_psuedo_data.shape,Y_psuedo_labels.shape) 

In [ ]:

# down load VGG 16 model to use in rotnet
image_input = Input(shape=(224, 224, 3))
model = VGG16(input_tensor=image_input, include_top=True,weights='imagenet')
model.summary()

In [ ]:
########   Model structured as per our class size  ###################################
last_layer = model.get_layer('fc2').output
#x= Flatten(name='flatten')(last_layer)
out = Dense(num_classes, activation='softmax', name='output')(last_layer)
custom_vgg_model = Model(image_input, out)
custom_vgg_model.summary()

for layer in custom_vgg_model.layers[:-1]:
    layer.trainable = False

custom_vgg_model.layers[3].trainable

custom_vgg_model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
custom_vgg_model.summary()

####################################################################################################################

In [ ]:
############################## Train the model with pseduo labels ######################################
X = X_psuedo_data
Y = Y_psuedo_labels
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.5, random_state=2)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)
hist = custom_vgg_model.fit(X_train, Y_train, batch_size=32, epochs=10, verbose=1, validation_data=(X_test, Y_test))
(loss, accuracy) = custom_vgg_model.evaluate(X_test, Y_test, batch_size=32, verbose=1)

In [ ]:
############################## Save the trained model ######################################
custom_vgg_model.save('RotNet_model')

################################ upto this is for problem 1 #####################################

## Now strat for problem 2 #####################################

In [ ]:
############################## Load the trained model ######################################
from tensorflow import keras
my_model = keras.models.load_model('RotNet_model')

In [ ]:
class_len = 4
num_classes = class_len
X = img_data
Y = np_utils.to_categorical(labels, num_classes)
print(X.shape,Y.shape) 

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=2)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)
my_model.summary()

In [ ]:
from keras.models import Model
layer_name = 'block5_pool'
model2 = Model(inputs= my_model.input, outputs=my_model.get_layer(layer_name).output)
model2.trainable = False
model2.summary()

In [ ]:
####### add some layer after pooling layers and take prediction layer as per the number of classes #####
from tensorflow.keras import layers, models
flatten_layer = layers.Flatten()
dense_layer_1 = layers.Dense(50, activation='relu')
dense_layer_2 = layers.Dense(20, activation='relu')
prediction_layer = layers.Dense(num_classes, activation='softmax')
model_super = models.Sequential([
    model2,
    flatten_layer,
    dense_layer_1,
    dense_layer_2,
    prediction_layer
])

model_super.summary()

In [ ]:
model_super.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)

In [ ]:
############### Train the model ##################################
hist = model_super.fit(X_train, Y_train, batch_size=32, epochs=5, verbose=1, validation_data=(X_test, Y_test))
#print('Training time: %s' % (t - time.time()))
(loss, accuracy) = model_super.evaluate(X_test, Y_test, batch_size=32, verbose=1)

print("[INFO] loss={:.4f}, accuracy: {:.4f}%".format(loss,accuracy * 100))

In [ ]:
model_super.save('self-supervised_model')

############## Finally save the model ##################################